In [ ]:
#Phase 0: Telegram bot 

from telegram import Update, ReplyKeyboardMarkup
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes


# Command
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    # Buttons:
    keyboard = [
        ["مشاوره درباره کتاب", "قوانین کتابخانه"]
    ]
    reply_markup = ReplyKeyboardMarkup(keyboard, resize_keyboard=True)

    await update.message.reply_text(
        "سلام! به ربات مشاور کتابخانه مرکزی خوش اومدی.\n"
        "یکی از گزینه‌های زیر رو انتخاب کن:",
        reply_markup=reply_markup
    )

# Handle Messages
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    text = update.message.text

    if text == "مشاوره درباره کتاب":
        await update.message.reply_text("به بخش مشاوره درباره کتاب خوش آمدی، چطور میتونم کمک کنم؟")
    elif text == "قوانین کتابخانه":
        await update.message.reply_text("به بخش قوانین کتابخانه خوش آمدی، چطور میتونم کمک کنم؟")
    else:
        await update.message.reply_text("لطفا یکی از گزینه‌ها رو انتخاب کن.")

# Main
def main():
    app = Application.builder().token(TOKEN).build()

    app.add_handler(CommandHandler("start", start))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    print("Bot is running...")
    app.run_polling()

if __name__ == "__main__":
    main()


In [ ]:
# Phase 1: Preprocessing on data
import os
import re
import pandas as pd


# Arabic → Persian replacements
ARABIC_TO_PERSIAN = {
    "ي": "ی",
    "ك": "ک",
    "ة": "ه",
    "ؤ": "و",
    "إ": "ا",
    "أ": "ا",
    "ئ": "ی"
}

# Arabic diacritics (harakat)
DIACRITICS_PATTERN = re.compile(r"[\u0610-\u061A\u064B-\u065F\u06D6-\u06DC\u06DF-\u06E8\u06EA-\u06ED]")

# Unwanted symbols: zero-width chars, bullets, arrows, dashes, underscores, quotes, etc.
EXTRA_SYMBOLS_PATTERN = re.compile(r"[\u200c\u200f\u200e◀▶■▪•\-\_\~\*\#\@\%\^\=\+\|\[\]\{\}\"“”‘’«»<>]")


def normalize_text(text: str) -> str:
    if not isinstance(text, str):
        return text

    # Arabic → Persian mapping
    for ar, fa in ARABIC_TO_PERSIAN.items():
        text = text.replace(ar, fa)

    # Remove Tatweel
    text = text.replace("ـ", "")

    # Remove diacritics
    text = DIACRITICS_PATTERN.sub("", text)

    # Remove special symbols (ZWNJ, bullets, dashes, underscores, quotes, etc.)
    text = EXTRA_SYMBOLS_PATTERN.sub(" ", text)

    # Normalize spaces
    text = re.sub(r"\s+", " ", text).strip()

    # Lowercase (for English)
    text = text.lower()

    return text


def is_persian_text(text: str) -> bool:
    """Check if text contains Persian characters."""
    if not isinstance(text, str):
        return False
    return bool(re.search(r"[\u0600-\u06FF]", text))


# Main processing
def normalize_and_separate_excels(input_folder: str, persian_output: str, english_output: str):
    persian_dfs = []
    english_dfs = []

    for file in os.listdir(input_folder):
        if file.endswith(".xlsx"):
            file_path = os.path.join(input_folder, file)
            print(f"Processing: {file_path}")

            df = pd.read_excel(file_path)

            # Normalize all columns
            for col in df.columns:
                df[col] = df[col].apply(normalize_text)

            # Detect if file is Persian or English
            sample_text = " ".join(df.astype(str).fillna("").values.flatten()[:50])
            if is_persian_text(sample_text):
                persian_dfs.append(df)
                print(" → Detected as Persian file.")
            else:
                english_dfs.append(df)
                print(" → Detected as English file.")

    # Combine and save Persian files
    if persian_dfs:
        combined_persian = pd.concat(persian_dfs, ignore_index=True)
        combined_persian.to_excel(persian_output, index=False)
        print(f"\nAll Persian Excel files combined and saved to: {persian_output}")
    else:
        print("No Persian Excel files found.")

    # Combine and save English files
    if english_dfs:
        combined_english = pd.concat(english_dfs, ignore_index=True)
        combined_english.to_excel(english_output, index=False)
        print(f"All English Excel files combined and saved to: {english_output}")
    else:
        print("No English Excel files found.")


# Run


if __name__ == "__main__":
    input_folder = "excel_files"  # Folder containing your .xlsx files
    persian_output = "final_persian_normalized.xlsx"
    english_output = "final_english_normalized.xlsx"

    normalize_and_separate_excels(input_folder, persian_output, english_output)


In [ ]:
#Phase 2: Doing embedding/chunks/... -> finishes with RAG

import pandas as pd
import numpy as np
import sqlite3
import faiss

from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

# -------------------------------------------------------------------
#  تنظیمات
# -------------------------------------------------------------------

SQLITE_PATH = "metadata.db"
FAISS_DIM = 1536  # ابعاد خروجی مدل embedding (مثلاً 1536 برای OpenAI)

# -------------------------------------------------------------------
#  ۱. خواندن و پیش‌پردازش داده
# -------------------------------------------------------------------

#df = pd.read_excel(EXCEL_PATH) The Output of the first phase in given as the input of the second phase.
df = pd.read_excel("final_normalized.xlsx")

# ستون شرح ترکیبی
df['شرح_کتاب'] = (
    df['عنوان'].fillna('') +
    " — " + df['موضوع'].fillna('') +
    " | " + df['رده اصلي'].fillna('')
)

# تبدیل به رکوردها
records = df.to_dict(orient="records")

# -------------------------------------------------------------------
#  ۲. تقسیم به چانک
# -------------------------------------------------------------------
splitter = CharacterTextSplitter(chunk_size=1500, chunk_overlap=100)

chunks = []
for rec in records:
    for txt in splitter.split_text(rec['شرح_کتاب']):
        chunks.append({
            "metadata": {
                "id": rec['رديف'],
                "title": rec['عنوان'],
                "author": rec['نويسنده'],
                "category": rec['رده اصلي'],
                "subject": rec['موضوع'],
                "publisher": rec['ناشر'],
                "year": rec['تاريخ'],
            },
            "text": txt
        })

# -------------------------------------------------------------------
#  ۳. تولید دسته‌ای (batch) embeddings
# -------------------------------------------------------------------
emb_client = OpenAIEmbeddings(openai_api_key=API_KEY)
all_texts = [c["text"] for c in chunks]
vectors = emb_client.embed_documents(all_texts)  # یک درخواست برای همه چانک‌ها

# تبدیل به آرایهٔ numpy
vecs = np.array(vectors, dtype='float32')
ids  = np.array([c["metadata"]["id"] for c in chunks], dtype='int64')

# -------------------------------------------------------------------
#  ۴. ساخت و پر کردن ایندکس FAISS
# -------------------------------------------------------------------
index = faiss.IndexFlatL2(FAISS_DIM)
index = faiss.IndexIDMap(index)
index.add_with_ids(vecs, ids)

# -------------------------------------------------------------------
#  ۵. درج متادیتا به‌صورت Bulk در SQLite
# -------------------------------------------------------------------
conn   = sqlite3.connect(SQLITE_PATH)
cursor = conn.cursor()

# ایجاد جدول اگر وجود ندارد
cursor.execute("""
CREATE TABLE IF NOT EXISTS metadata (
    id        INTEGER PRIMARY KEY,
    title     TEXT,
    author    TEXT,
    category  TEXT,
    subject   TEXT,
    publisher TEXT,
    year      TEXT
)
""")

# آماده‌سازی داده‌ها برای درج
to_insert = [
    (
        c["metadata"]["id"],
        c["metadata"]["title"],
        c["metadata"]["author"],
        c["metadata"]["category"],
        c["metadata"]["subject"],
        c["metadata"]["publisher"],
        c["metadata"]["year"],
    )
    for c in chunks
]

# درج دسته‌ای
cursor.executemany("""
    INSERT OR REPLACE INTO metadata
    (id, title, author, category, subject, publisher, year)
    VALUES (?, ?, ?, ?, ?, ?, ?)
""", to_insert)

conn.commit()
conn.close()

print(f"⚡️ آماده! تعداد چانک‌ها: {len(chunks)}, ابعاد FAISS: {FAISS_DIM}")


⚡️ آماده! تعداد چانک‌ها: 117404, ابعاد FAISS: 1536


In [ ]:
chunks

[{'metadata': {'id': 1,
   'title': nan,
   'author': '/ علي ذاكرالحسيني، احسان ملكيان',
   'category': 'TK',
   'subject': 'شبكه هاي كامپيوتري -- اقدامات تأميني ◄ كامپيوترها -- ايمني اطلاعات ◄ اينترنت -- اقدامات تأميني',
   'publisher': 'تهران : نص ، \u202e\u202d1387\u202c.',
   'year': nan},
  'text': '— شبكه هاي كامپيوتري -- اقدامات تأميني ◄ كامپيوترها -- ايمني اطلاعات ◄ اينترنت -- اقدامات تأميني | TK'},
 {'metadata': {'id': 2,
   'title': nan,
   'author': '/ مولف ماري برامبرگ ، جوليوس ليب، ترجمه رضا دانشور',
   'category': 'PE',
   'subject': 'زبان انگليسي -- واژگان -- مسائل، تمرين ها و غيره',
   'publisher': 'تهران : : جنگل، : جاودانه ، ، \u202e\u202d1387\u202c.',
   'year': '1387\u202c'},
  'text': '— زبان انگليسي -- واژگان -- مسائل، تمرين ها و غيره | PE'},
 {'metadata': {'id': 3,
   'title': nan,
   'author': '/گردآوري مهدي غضنفري ، سميه عليزاده، بابك تيمورپور.',
   'category': 'QA',
   'subject': 'داده كاوي',
   'publisher': 'تهران : دانشگاه علم و صنعت ايران \u202b، 1387.\u202

In [ ]:
def embed_query(query: str):
    # embed_documents هم می‌تواند استفاده شود؛ embed_query برای یک رشته مناسب است
    return emb_client.embed_query(query)


In [ ]:
import numpy as np

def search_faiss(q_vec: np.ndarray, k: int = 5):
    # FAISS L2 search
    distances, indices = index.search(np.array([q_vec], dtype='float32'), k)
    return indices[0]  # آرایه‌ای از idهای متادیتا


In [ ]:
def fetch_metadata(ids: list[int]):
    placeholders = ",".join("?" for _ in ids)
    conn = sqlite3.connect(SQLITE_PATH)
    cursor = conn.cursor()
    cursor.execute(f"SELECT * FROM metadata WHERE id IN ({placeholders})", tuple(ids))
    rows = cursor.fetchall()
    conn.close()
    # برمی‌گرداند لیستی از دیکشنری‌ها
    return [
        {
            "id": r[0],
            "title": r[1],
            "author": r[2],
            "category": r[3],
            "subject": r[4],
            "publisher": r[5],
            "year": r[6],
        }
        for r in rows
    ]


In [ ]:
from openai import OpenAI
import numpy as np
import sqlite3

client = OpenAI(api_key=API_KEY)

def rag_answer(user_query: str, k: int = 5) -> str:
    # ۱. embed پرسش
    q_vec = emb_client.embed_query(user_query)
    
    # ۲. جستجو در FAISS
    distances, indices = index.search(
        np.array([q_vec], dtype='float32'), k
    )
    ids = [int(i) for i in indices[0] if i != -1]
    
    # ۳. اگر چیزی نیافتیم
    if not ids:
        return "متأسفم، اطلاعات کافی در مجموعه‌ی شما نیست."
    
    # ۴. واکشی متادیتا
    placeholders = ",".join("?" for _ in ids)
    conn = sqlite3.connect(SQLITE_PATH)
    cursor = conn.cursor()
    cursor.execute(
        f"SELECT id, title, subject, author, year FROM metadata WHERE id IN ({placeholders})",
        tuple(ids)
    )
    rows = cursor.fetchall()
    conn.close()
    
    # ۵. ساخت Context با ارجاع به شناسه‌ها
    context_parts = [
        f"[{r[0]}] «{r[1]}» — موضوع: {r[2]} (نویسنده: {r[3]}, سال: {r[4]})"
        for r in rows
    ]
    context = "\n".join(context_parts)
    
    # ۶. آماده‌سازی پیام‌ها
    system_msg = """
    شما دستیار کتاب‌شناس هستید. 
    فقط می‌توانید از فهرست کتاب‌هایی که در دیتابیس محلی موجود است استفاده کنید. 
    اگر نمی‌توانید جواب را دقیقاً از آن استخراج کنید، بگویید:
    «متأسفم، اطلاعات کافی در مجموعه‌ی شما نیست.»
    """
    user_msg = (
        f"با توجه به اطلاعات زیر (فقط این کتاب‌ها را در نظر بگیرید):\n{context}\n\n"
        f"سؤال: «{user_query}»\n\n"
        "لطفاً در پاسخ ارجاع دهید به شناسه‌ی کتاب‌ها (مثلاً [12]) و از هیچ منبع دیگری استفاده نکنید."
    )
    
    # ۷. فراخوانی مدل
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user",   "content": user_msg}
        ]
    )
    return response.choices[0].message.content


In [ ]:
print(rag_answer("کتاب های مرتبط با HTML را معرفی کن"))

کتاب‌های مرتبط با HTML عبارتند از:

1. [56820] «راهنماي جامع ‮‭HTML - DHTML - XML‬»
2. [56821] «راهنماي جامع ‮‭HTML‬ [اچ. تي. ام. ال] و ‮‭Dynamic HTML‬ [دايناميك اچ. تي. ام. ال]»
3. [87181] «كتاب آموزشي‮‭HTML &amp; XHTML‬[اچ. تي. ام. ال &amp; ايكس. اچ. تي. ام. ال] در ‮‭۲۴‬ ساعت»
4. [87182] «كتاب آموزشي‮‭HTML XHTML‬[اچ. تي. ام. ال اند ايكس. اچ. تي. ام. ال] در ‮‭۲۴‬ ساعت»
5. [101188] «مرجع كامل ‮‭HTML‬ و اصول طراحي وب»


In [ ]:
#Phase 3: Connection Metadata to an AI telegram bot -> in order to it's answer, mathces Data of library.



